In [1]:
import pandas as pd

In [2]:
data_dir = "./data"
file_name="Copy of Briggs Course Prerequisites 201710 201720 13MAR2018.xls"

In [3]:
df = pd.read_excel("{0}/{1}".format(data_dir, file_name), sheet_name="Original Sheet")
df['PRE_REQUISITES'] = df['PRE_REQUISITES'].fillna("")

In [4]:
sample_string = "(Undergraduate level AGBU 1005 Minimum Grade of D or Undergraduate level AM 15A Minimum Grade of D) and (Undergraduate level AGBU 1006 Minimum Grade of D or (Undergraduate level AM 15B Minimum Grade of D) and Undergraduate level AGRI 1004 Minimum Grade of D or Undergraduate level AG 15B Minimum Grade of D)"

## Checking for Complicated cases to make determination about validity of derived solution

In [5]:
df['ANDs'] = df['PRE_REQUISITES'].str.lower().str.split(" and ").apply(len)
df['ORs'] = df['PRE_REQUISITES'].str.lower().str.split(" or ").apply(len)

In [54]:
high_and = df['ANDs'] > 5
high_or = df['ORs'] > 5
df[high_and & high_or].sort_values(["ORs","ANDs"],ascending=False).head()


,TERM_CODE,TERM_DESC,SUBJ_CODE,CRSE_NUMB,SECTION,COURSE_FACULTY,COURSE_DEPARTMENT,PRE_REQUISITES,ANDs,ORs
770,201710,2017/2018 Semester I,ECON,2000,E01,Social Sciences,Economics,(Undergraduate level ECON 1001 Minimum Grade o...,6,38
784,201710,2017/2018 Semester I,ECON,2015,E01,Social Sciences,Economics,(Undergraduate level ECON 1003 Minimum Grade o...,6,38
772,201710,2017/2018 Semester I,ECON,2000,S01,Social Sciences,Economics,(Undergraduate level ECON 1001 Minimum Grade o...,7,35
786,201710,2017/2018 Semester I,ECON,2015,S01,Social Sciences,Economics,(Undergraduate level ECON 1004 Minimum Grade o...,6,35
775,201710,2017/2018 Semester I,ECON,2000,SAL,Social Sciences,Economics,(Undergraduate level ECON 1001 Minimum Grade o...,6,30


In [55]:
subject = df['SUBJ_CODE'] == "NURS"
code = df['CRSE_NUMB'] == 3013
text = list(df[subject & code]['PRE_REQUISITES'])[0]

In [56]:
print(len(text.split("(")))
print(len(text.split(")")))

13
13


In [68]:
def count_prerequisites(text):
    components = text.split("(")
    joiners = [] # used to keep track of whether an and or an or is used to join the sections
    count_sec = [] # used to keep track of prerequisites within each section
    for sec_id in range(1, len(components)):
        sec = components[sec_id]
        end = sec.split(")")
        if len(end) > 1:
            joiners.append(end[-1].strip())
        count_sec.append(len(end[0].split(" and ")))
    # print(text)
    # print("Joiners has a length of {0} and contents {1}".format(len(joiners),joiners))
    # print("count_sec has a length of {0} and contents {1}".format(len(count_sec),count_sec))
    # print(len(components[1].split(")")[0].split(" and ")))
    total = 0
    if len(count_sec) > 0:
        total = count_sec[0]
        for i in range(0, len(joiners) - 1):
            if joiners[i] == "and":
                total += count_sec[i+1]
            elif joiners[i] == "or":
                if count_sec[i+1] > total:
                    total = count_sec[i+1]

    # print("Total prerequisites is {0}".format(total))
    return total


In [69]:
df[high_and & high_or].sort_values(["ORs","ANDs"],ascending=False).head()

,TERM_CODE,TERM_DESC,SUBJ_CODE,CRSE_NUMB,SECTION,COURSE_FACULTY,COURSE_DEPARTMENT,PRE_REQUISITES,ANDs,ORs
770,201710,2017/2018 Semester I,ECON,2000,E01,Social Sciences,Economics,(Undergraduate level ECON 1001 Minimum Grade o...,6,38
784,201710,2017/2018 Semester I,ECON,2015,E01,Social Sciences,Economics,(Undergraduate level ECON 1003 Minimum Grade o...,6,38
772,201710,2017/2018 Semester I,ECON,2000,S01,Social Sciences,Economics,(Undergraduate level ECON 1001 Minimum Grade o...,7,35
786,201710,2017/2018 Semester I,ECON,2015,S01,Social Sciences,Economics,(Undergraduate level ECON 1004 Minimum Grade o...,6,35
775,201710,2017/2018 Semester I,ECON,2000,SAL,Social Sciences,Economics,(Undergraduate level ECON 1001 Minimum Grade o...,6,30


In [74]:
df['#PRE_REQUISITES'] = df['PRE_REQUISITES'].str.lower().apply(count_prerequisites)

In [76]:
df.to_excel("./data/output.xlsx")